In [4]:
import os
import sqlite3


# PATH = os.environ.get("DATA_PATH")
# PATH = "var/lib/data"

PATH = './data'

DATABASE_PATH = os.path.join(os.path.abspath(PATH), 'financedata.db')

connection = sqlite3.connect(DATABASE_PATH, check_same_thread=True)

cursor = connection.cursor()

In [5]:
ticker = "MSFT"

cik = cursor.execute(f"""
    SELECT * FROM companyInfo
    WHERE cik=320193
               """).fetchall()

cik

[(320193,
  'AAPL',
  'Apple Inc.',
  'operating',
  3571,
  'Electronic Computers',
  'Nasdaq',
  '',
  '',
  'Large accelerated filer',
  '',
  928,
  'CA')]

In [15]:
cursor.execute(f"""
    SELECT cik FROM companyInfo
    WHERE ticker="MSFT"
               """).description

(('cik', None, None, None, None, None, None),)

In [9]:
cursor.execute(f"""
    SELECT * FROM submissionForm
               """).fetchmany(2)


[('0000851968-24-000082',
  851968,
  '2024-04-26',
  '2024-03-30',
  '2024-04-26T12:08:53.000Z',
  34,
  '10-Q',
  '001-13697',
  24880651,
  '',
  8430169,
  1,
  1,
  'mhk-20240330.htm',
  '10-Q'),
 ('0000851968-24-000027',
  851968,
  '2024-02-23',
  '2023-12-31',
  '2024-02-23T17:16:41.000Z',
  34,
  '10-K',
  '001-13697',
  24672610,
  '',
  14837980,
  1,
  1,
  'mhk-20231231.htm',
  '10-K')]